# Использование хаба промптов для суммаризации книг
В качестве примера суммаризируем первые две главы книги Булгакова "Мастер и Маргарита".

## Установка необходимых библиотек

In [1]:
!pip install langchain_gigachat python_dotenv --quiet -U


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Инициализация GigaChat
> Обратите внимание, что нужно использовать большую модель гигачат, поддерживающую 8 тысяч токенов и более!
> Обратите внимание, что если в книге могут встречаться слова и темы, которые ГигаЧат может посчтитать неоднозначными,
> следует использова флаг отключения фильтрации - profanity=False. По возможности использования этого флага следует обратиться 
> в поддержку GigaChat. На данный момент эта опция доступна только юридическим лицам.

In [ ]:
from langchain.prompts import load_prompt
from langchain.chains.summarize import load_summarize_chain
from langchain_gigachat import GigaChat

giga = GigaChat(credentials="ВАШИ АВТОРИЗАЦИОННЫЕ ДАННЫЕ", model="GigaChat-2-Max", verify_ssl_certs=False, timeout=600)

## Суммаризация книг, больших текстов
### Загрузка и подготовка данных

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Укажите полный путь до файла (зависит от окружения)
loader = TextLoader("../../../../docs/docs/cookbook/мастер_и_маргарита.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 7000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
documents = text_splitter.split_documents(documents)
print(f"Количество частей книги: {len(documents)}")

Количество частей книги: 11


### Создание цепочки суммаризации

In [18]:
book_map_prompt = load_prompt("summarize_book_map.yaml")
book_combine_prompt = load_prompt("summarize_book_combine.yaml")

chain = load_summarize_chain(giga, chain_type="map_reduce", 
                             map_prompt=book_map_prompt,
                             combine_prompt=book_combine_prompt,
                             verbose=False)

In [19]:
res = chain.invoke({"input_documents": documents})

In [20]:
print(res["output_text"].replace(". ", ".\n"))

Роман Михаила Булгакова «Мастер и Маргарита» открывается мистической встречей редактора Массолита Берлиоза и поэта Ивана Бездомного с иностранным гостем Воландом на Патриарших прудах.
Во время диалога о существовании Иисуса Христа, Воланд пророчит скорую гибель Берлиозу, предвещая последующие магические события.
Параллельно автор переносит нас в Древний Иерусалим, где прокуратор Понтий Пилат судит арестованного бродягу-философа Иешуа Га-Ноцри, несмотря на личную симпатию, вынужден утверждать несправедливый смертный приговор.

Иешуа предстает мудрым и добрым человеком, видящим истину даже в сердце сурового правителя.
Его учение о любви и всепрощении поражает Пилата, оставляя глубокий след в душе последнего.
В итоге внутреннее противоречие вынуждает Пилата отдать распоряжение о распятии невиновного, смирившись с собственным бессилием и чувством вины.

События возвращаются в Москву, где продолжаются сатирически изображённые сцены абсурдной реальности советского быта и трагикомичные послед

Если нужно изменить количество предложений в суммаризации, то с помощью дополнительных параметров можно задать нужные объемы текста на этапах map и reduce.

In [21]:
res = chain.invoke({"input_documents": documents, "map_size": "одно предложение", "combine_size": "три предложения"})

In [22]:
print(res["output_text"].replace(". ", ".\n"))

Роман Михаила Булгакова «Мастер и Маргарита» открывается сценой знакомства читателей с главным антагонистом — демоном Воландом, неожиданно появляющимся на Патриарших прудах и вступающим в философско-теологический диспут с редактором Берлиозом и поэтом Иваном Бездомным.
Диалог перерастает в глубокий спор о Боге, истине и предопределении судеб, причём загадочные намёки Воланда сопровождаются зловещими предсказаниями гибели главных персонажей.

Параллельно развивается сюжет, связанный с древнееврейской историей времен правления Римского императора Тиберия, когда известный римский прокуратор Понтий Пилат допрашивает арестованного проповедника Иешуа (Иисуса), которого ложно обвинили в подстрекательстве к мятежу.
В ходе расследования становится ясно, что единственный свидетель обвинения Левий Матвей исказил речь Иешуа, ошибочно приписывая ему призыв разрушить храм Иерусалима.


Несмотря на внутренние колебания и желание следовать справедливому решению, Понтий Пилат выносит жестокое судебное 